In [42]:
from sentence_transformers import SentenceTransformer, InputExample
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator

In [43]:
import pandas as pd
import torch
from torch.utils.data import DataLoader
from transformers import BertTokenizer, BertModel
import torch.nn.functional as F
from sentence_transformers import util, losses
import numpy as np
from sentence_transformers import SentenceTransformer, InputExample
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
import xgboost as xgb
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [44]:
fine_tuning_file_dir = './data/computers_train/computers_medium_large.json.gz'
ft_valid_file_dir = './data/computers_valid/computers_valid_medium.csv'
n_epochs = 1

In [45]:
# finetuning bert model
large_df = pd.read_json(fine_tuning_file_dir, compression='gzip', lines=True)
large_df = large_df.drop(columns=['id_left', 'id_right', 'cluster_id_left', 'cluster_id_right', 'identifiers_left', 'identifiers_right'])
valid_ids = pd.read_csv(ft_valid_file_dir)
ft_train_df = large_df.loc[~large_df['pair_id'].isin(valid_ids['pair_id'])]

FileNotFoundError: File ./data/computers_train/computers_medium_large.json.gz does not exist

In [46]:


def get_similarity(model, s_pair):
    if s_pair[0] == None or s_pair[1] == None:
        return None 

    if s_pair[0] == s_pair[1]:
        return 1.0
    
    embds = model.encode(s_pair)
    sim_score = util.cos_sim(embds[0], embds[1]).item()
    return sim_score

def get_similarity_keyval(dic1, dic2):
    key_set = set()
    tot_sim = 0
    for key, value1 in zip(dic1.keys(), dic1.values()):
        if key in dic2:
            value2 = dic2[key]
            if value2 == value1:
                tot_sim += 1

        key_set.add(key)

    for key2 in dic2:
        key_set.add(key2)

    return tot_sim / len(key_set)

In [48]:
data_df = pd.read_json('./data/computers_train/computers_train_medium.json.gz', compression='gzip', lines=True)
data_df = data_df.drop(columns=['id_left', 'id_right', 'cluster_id_left', 'cluster_id_right', 'identifiers_left', 'identifiers_right'])

valid_ids = pd.read_csv('./data/computers_valid/computers_valid_medium.csv')

train_df = data_df.loc[~data_df['pair_id'].isin(valid_ids['pair_id'])]
valid_df = data_df.loc[data_df['pair_id'].isin(valid_ids['pair_id'])]

In [55]:
idx = 0
pair_no = 0
idx_to_pair_no = {}
description_list= []
for left, right in zip(train_df['description_left'], train_df['description_right']):
    if left != None and right != None:
        pair_no += 1
        idx_to_pair_no[idx] = pair_no
        description_list.append(left)
        description_list.append(right)
    idx += 1

In [57]:
len(description_list)

8290

In [58]:
sbert_model_eng.encode(description_list)

array([[ 1.20874070e-01,  1.69541880e-01,  1.97632685e-01, ...,
         3.76053620e-03, -1.01542033e-01,  9.13804471e-02],
       [ 1.13327131e-01,  1.18805088e-01,  1.91182092e-01, ...,
        -2.79869908e-03, -1.02162220e-01,  5.41441888e-02],
       [-9.08283368e-02,  3.79040428e-02,  1.22852005e-01, ...,
         8.88176113e-02,  3.68592963e-02,  2.26478856e-02],
       ...,
       [-6.76703528e-02,  2.83360690e-01, -9.54265371e-02, ...,
         8.91093984e-02, -2.48319328e-01,  2.23582402e-01],
       [ 6.41614869e-02,  1.75788209e-01, -2.01825052e-01, ...,
        -6.47189663e-05, -2.96221256e-01,  2.78991967e-01],
       [-6.76703528e-02,  2.83360690e-01, -9.54265371e-02, ...,
         8.91093984e-02, -2.48319328e-01,  2.23582402e-01]], dtype=float32)

In [62]:


def get_similarity_vectorizer(value_left, value_right, vectorizer):
    if value_left == None or value_right == None:
        return None

    if value_left == value_right:
        return 1.0
    tf_idf_left = vectorizer.transform([value_left])
    tf_idf_right = vectorizer.transform([value_right])
    sim = cosine_similarity(tf_idf_left, tf_idf_right)[0][0]
    return sim


def compute_features(df):
    # TODO: title, brand and keyValue using Tfidf / encode all sentences at once to sbert
    num_df = pd.DataFrame()
    text_df = pd.concat([df['title_left'], df['title_right']])
    vectorizer = TfidfVectorizer()
    vectorizer = vectorizer.fit(text_df.values)
    num_df['title_sim'] = df.apply(
        lambda x: get_similarity_vectorizer(x['title_left'], x['title_right'], vectorizer), axis=1)
    print("title done")

    text_df = pd.concat([df['description_left'], df['description_right']])
    text_df = text_df.dropna(how="any", axis=0)
    vectorizer = TfidfVectorizer()
    vectorizer = vectorizer.fit(text_df.values)
    num_df['description_sim'] = df.apply(
        lambda x: get_similarity_vectorizer(x['description_left'], x['description_right'], vectorizer), axis=1)
    print("descrition done")

    text_df = pd.concat([df['brand_left'], df['brand_right']])
    text_df = text_df.dropna(how="any", axis=0)
    vectorizer = TfidfVectorizer()
    vectorizer = vectorizer.fit(text_df.values)
    num_df['brand_sim'] = df.apply(lambda x: get_similarity_vectorizer(x['brand_left'], x['brand_right'], vectorizer), axis=1)

    num_df['label'] = df['label']
    return num_df


In [63]:
num_train_df = compute_features(train_df)

title done
descrition done


In [64]:
num_valid_df = compute_features(valid_df)

title done
descrition done


In [65]:
num_train_df

,title_sim,description_sim,brand_sim,label
3,0.574667,NaN,NaN,1
4,0.586431,0.039588,1.0,1
5,0.641244,0.140730,NaN,1
6,0.662503,NaN,NaN,1
7,0.212899,NaN,NaN,1
...,...,...,...,...
8089,0.144609,0.109454,0.0,0
8090,0.000000,0.000000,0.0,0
8091,0.295581,0.361471,1.0,0
8092,0.343734,0.207120,1.0,0


In [66]:
#importing dataset
#isolate the x and y variables
y_train = np.array(num_train_df['label'])
X_train = num_train_df.drop(columns=["label"])
    
y_test = np.array(num_valid_df['label'])
X_test = num_valid_df.drop(columns=["label"])


In [67]:

#create xgboost matrices
Train = xgb.DMatrix(X_train, label = y_train)
Test = xgb.DMatrix(X_test, label = y_test)

#set the parameters
parameters = {'learning_rate': 0.3,
               'max_depth': 2,
               'colsample_bytree': 1,
               'subsample': 1,
               'min_child_weight': 1,
               'gamma': 0, 
               'random_state': 1500,
               'eval_metric': "auc",
               'objective': "binary:logistic"}


model = xgb.train(params = parameters,
                dtrain = Train,
                num_boost_round = 200,
                evals = [(Test, "Yes")],
                verbose_eval = 50)


#PRedictions
pred = model.predict(Test)
pred = np.where(pred > 0.5, 1, 0)

confusion_matrix = confusion_matrix(y_test, pred)
report = classification_report(y_test, pred)
print(report)


[0]	Yes-auc:0.87963
[50]	Yes-auc:0.91091
[100]	Yes-auc:0.91172
[150]	Yes-auc:0.90917
[199]	Yes-auc:0.90705
              precision    recall  f1-score   support

           0       0.91      0.94      0.93      1267
           1       0.77      0.67      0.72       352

    accuracy                           0.88      1619
   macro avg       0.84      0.81      0.82      1619
weighted avg       0.88      0.88      0.88      1619

